# CarND-Controls-PID Writeup
Self-Driving Car Engineer Nanodegree Program 

[![Udacity - Self-Driving Car NanoDegree](https://s3.amazonaws.com/udacity-sdc/github/shield-carnd.svg)](http://www.udacity.com/drive)


## Overview

-------------------------------------------------

This System Integration project implements a software suite that could drive an autonomous vehicle in the [simulator](https://github.com/udacity/self-driving-car-sim/releases) or in real world with some limitations (e.g. in a controlled safe environment).

The system is based on the robotic operational system [ROS](https://www.ros.org/) and contains an ensemble of ROS nodes, performing various tasks for controlling the vehicle:

![architecture](./imgs/final-project-ros-graph-v2.png "Vehicle control system ROS nodes")

#### Nodes

###### Waypoint loader

Loads base waypoints from a CSV file. During a real world driving session, the base waypoints would be generated by a route planner using some kind of maps such as Google, OSM, etc. In scope of this project there are a set of predefined waypoints around the circular track of the simulator scene. The loaded waypoints are being published into the `/base_waypoints` topic. Also the waypoint loader sets the desired vehicle speed.
**Note:** I am not sure if a student is supposed to edit this node, however I was not able to fullfill the "navigate the full track more than once" requirement of the rubric without changing the waypoint loading algorithm. I have disabled the deceleration at the end of the base waypoint list.

###### Waypoint updater

This node uses the base waypoints from the `/base_waypoints` topic, and also information about the current position and velocity of the vehicle, and some information about surrounding from, respectively, `/current_pose` and `/traffic_waypoint` topics. Based on the desired route (base wapoints), current vehicle position and the surroundungs, this node generates a list of updated waypoints to drive safely and comfortably. These "final" waypoints most of the time are just copies of the base ones, but might have different desired speed and coordinates in case of an obstacle is ahead or a traffic light demands to stop. THe final waypoints are then being published to the `/final_waypoints` topic

###### Waypoint follower

To follow the final waypoints, the vehicle must operate its steering and brake/throttle. The waypoint follower node generates corresponding commands to the vehicle driving circuits, using the current vehicle position and velocities and the final waypoint list. The commands are encoded in TwistStamped format and being published into the `/twist_cmd` topic.

###### Drive-by-wire (DBW) node and twist controller

This node implements the practical control of the vehicle's steering, throttle and brake, according to the twist commands, generated by the waypoint follower. The node gathers information about current vehicle state from the `/current_velocity` topic and takes a command from the `/twist_cmd` topic. Then, using a PID controller, a particular value of the throttle/brake is generated to maintain desired speed, and another PID controller provides a particular value of a steering angle to reach the desired angular velocity and/or car heading. The node operates (e.g. sends steering, throttle, brake commands to, respectively, `/vehicle/steering_cmd`, `/vehicle/throttle_cmd`, `/vehicle/brake_cmd`) only if there has been an enabling message published into `/vehicle/dbw_enabled`.

###### Traffic light detector

This node takes a color image from the `/image_color` topic and detects if there is a traffic light in the image, and what is the state of the traffic light (red, green, yellow, ...). Also there is a helper information being published to the `/vehicle/traffic_lights` topis by the simulator, so during the driving in the simulated environment it is possible to validate the detected state of the traffic light.

###### Bridge/server or cppbridge

This node serves as a bridge between the simulator and other ROS nodes, or a bridge between the ROS nodes and a real vehicle CAN controlling device. Unfortunately, due to significant changes in Python packages during last years, I was not able to run the bridge server successfuly, so I have reimplemented this functionality in a `cppbridge` node, using C++ and uWS library. The corresponding code can be found at `cppbridge` branch of the repository, in the `ros/src/cppbridge/...` folders. *Note: this code is not compilable within the Udacity workspace, so I have removed it from the master/develop branches*. The cpp implementation does exactly the same as the original Python one, so I wouldn't review it here separately.
The bridge takes the commands from the `/vehicle/steering_cmd`, `/vehicle/throttle_cmd`, `/vehicle/brake_cmd` topics, waypoints info (to render them on the scene) from the `/final_waypoints` topic, and sends the corresponding TCP packets to the simulator. Also, the server receives TCP packets with the vehicle status and surroundings info, and then publishes this information into the `/vehicle/dbw_enabled`, `/current_pose`, `/current_velocity`, `/image_color` topics.


## Implementation details

-------------------------------------------------

### Traffic light detection
![](./imgs/tl-detector-ros-graph.png)

I have not implemented any neural network model, but instead developed a classic computer vision algorithm for detection of a traffic light state. The main reason is: there are many models that can detect the traffic light itself, such as MobilenetV1, YOLO, and others, so there is no need to reinvent the wheel here. Using such a pre-trained model, it is easy to aquire a bounding box around a traffic light, and, using the box as ROI, extract only the image of the traffic light itself.
The camera in the simulator is directed upright, so usually the traffic light is the most intensively coloured object on a sky or mountains background:

[tl_sim_example_1]: ./imgs/traffic/baseline/00004.jpg "Simulator traffic light example"
[tl_sim_example_2]: ./imgs/traffic/baseline/00046.jpg "Simulator traffic light example"
[tl_sim_example_3]: ./imgs/traffic/baseline/00029.jpg "Simulator traffic light example"

|![][tl_sim_example_1]|![][tl_sim_example_2]|![][tl_sim_example_3]|
|:-------------------:|:-------------------:|:-------------------:| 

This allows usage of a relatively simple approach to detect the state of the traffic light:
1. Find the most intensively colored pixels
2. Find the brightest pixels
3. Combine the most colored *AND* the brightest pixels, forming a mask of the traffic light candidates
4. Cut the rest pixels off the image
5. Measure, what is the dominant color of the kept pixels, and if it corresponds to one of the [red, yellow, green] - the job is done, otherwise there is probably no traffic light in the picture, or its state couldn't be detected.

The dominant color is measured using only non-black (nonzero) pixels of the masked image.

To tune some parameters of this algorithm (saturation and value thresholds, maximum levels, etc.) I have gathered several dosens of the images, published by the simulator to the `/image_color` ROS topic, and also have found a dosen of the real traffic lights photos from Internet.

The picture below shows (from left to right):
* the original camera image
* "saturation" channel of the camera image in HSV colorspace, thresholded
* "value" channel of the camera image in HSV colorspace, thresholded
* a weighted sum of the saturation and value channels
* a cutting mask, where white pixels indicates the brightest AND most intensively colored ones
* the resulting image, containing only the pixels of the traffic lights, and a detection label

![detection_examples](./imgs/illustrations.jpg "Detection examples")

This algorithm works not too bad even on the real-life traffic light images, if the traffic light itself is bright and saturated enough (with respect to its background). In the rows 14 and 16 of the image above (the last ones) there are misdetections, caused by presence of a large brightly colored areas along with the traffic light itself. With a camera, directed upwards, the problem, shown in row 16, should never happen. The problem in row 14 is very specific to a particular local laws, so unless a traffic light has a yellow bounding frame, the algorithm would still work fine.

I have experimented with a possible performance boost via using a resized image in the detection pipeline (150x100 or 300x200 instead of the original 800x600 the camera provides), but I have not note any significant change in performance; however, the detector was working worse with such a resize, because small colored dots (of a distant traffic light) were corrupted because of resizing.

The exact code with detailed comments can be found in `ros/src/tl_detector/light_classification/tl_classifier.py`


### Waypoint updater
![](./imgs/waypoint-updater-ros-graph.png)

As soon as there is a detected traffic light, the final waypoints can be updated. If there is a red traffic light ahead, the vehicle must smoothly decelerate and stop close to a corresponding stopline. If there is a yellow traffic light ahead, the vehicle should start deceleration as well, as in the provided environment the yellow light is always immediately followed by a red light. Also, the vehicle must decelerate if there is an obstacle ahead, however the obstacle handling is out of scope of this project.

To decelerate ant stop the vehicle, some of the final waypoints must have gradually decreasing (until zero) linear velocity but the waypoint updater does not have information about the current vehicle speed. So, instead of computing the speed difference between the current and desired speed, the waypoint updater computes only desired speed. The waypoint, that is the closest to a stopline, and all waypoints _after_ it (along the base waypoints predefined route direction) are assigned with a zero velocity. Each waypoint _back_ from the stopline waypoint is assigned with a linear velocity, proportional to the square root of a distance between this point and the stopline waypoint:

```
...
    dist = self.distance(final_candidates, i, waypoints_until_stop)
    new_desired_spd_ms = math.sqrt(2 * MAX_DECEL * dist)
...
```
This way, some of the waypoints would get a velocity that is higher than the allowed driving speed, but it is not a problem as the waypoint follower node would take care of keeping the speed limit.

The final waypoints are being updated with a frequency about 10Hz, and there are 100 waypoints generated in advance. I have performed several experiments using more waypoints and/or higher publishing frequency, but it only leads to more lags in the ROS functioning, so at the end of the day I have kept these values as the best.

The code with some extra comments can be found in `ros/src/waypoint_loader/waypoint_loader.py`.


### Twist controller
![](./imgs/dbw-node-ros-graph.png)

The twist controller class implements the functionality of the DBW node. It computes three control command values:

###### Throttle

The current linear velocity of the car is filtered with an instance of `LowPassFilter` in order to eliminate noise.
The throttle PID controller gets the difference between the (filtered) current vehicle linear velocity and the desired one (given in the corresponding point of the final waypoints). If the desired speed is higher than the current, it means the throttle should be positive, and the exact value (in range 0 to 1.0) is being computed via the PID algorithm. I have limited the maximum allowed throttle value to 0.6 to avoid any agressive acceleration from a full stop.
If the desired speed is less than the current one, but the difference is relatively small, the throttle should be set to 0, and the car would slowly decelerate due to friction and other energy dissipations.
This way, while driving without obstacles, the vehicle speed is maintained only using a throttle control (e.g. without brakes) which guarantees a comfort smooth longitudinal movement.

###### Brake

If the desired speed is less than the current one and the difference is relatively large (especially when the desired speed is 0), the deceleration via constant power dissipation with zero throttle is not useful, and the brake force is applied to the vehicle's wheels.
The brake torque is computed using a desired (safe) deceleration rate and the difference between velocities:
```
...
    deceleration = max(speed_delta_ms, self.decel_limit)
    self.brake_torque_nm = min(abs(deceleration) * self.vehicle_mass * self.wheel_radius,
                               self.fullstop_brake_torque_nm)
...
```
However, this dynamically changing torque value makes sense only during a smooth deceleration, when the car is still moving relatively fast.
Due to the automatic transmission, the vehicle needs some decent amount of brake torque to stand still, even when the throttle is zero.
So, in order to fully stop the car, that is already moving very slowly, the brake torque is just gradually increased:

```
while ...
    self.brake_torque_nm = min(self.brake_torque_nm + 10., 
                               self.fullstop_brake_torque_nm)
   ...
...
```
If the desired speed is larger than the current speed, the brake torque is always being set to 0.
                                           
###### Steering angle

The steering angle must be set to some value, that, considering current linear velocity of the vehicle, would cause rotation of the vehicle's heading to a direction, that leads to the closest waypoint ahead. The necessary angular velocity is computed by the waypoint follower node and posted as a twist command.
The exact value of the steering angle is then computed, based on the desired angular velocity and the current linear velocity of the vehicle, and also taking into account the wheel base af the car, by the `YawController` instance. As the current linear velocity is filtered, the yaw controller usually returns a decent angle value without unnecessary oscillations.


The code of the twist controller class can be found in `ros/src/twist_controller/twist_controller.py`.



## Known issues
-------------------------------------------------
#### Traffic light detection problems

Unfortunately, the traffic light detection algorithm, described above, does not work at all with the images from the Udacity Car bag:

[tl_bag_1]: ./imgs/traffic/bag/1.png
[tl_bag_2]: ./imgs/traffic/bag/2.png

|![][tl_bag_1]|![][tl_bag_2]|
|:-----------:|:-----------:|

Because the colors on the image are extremely bleached, the detection, based on colors, fails:

[bag_examples]: ./imgs/bag_samples.jpg "Detection examples"
![][bag_examples]

Anyway, in the simulator the detection accuracy is quite close to 100%. I have tested the algorithm on several laps, and there were only a couple of misdetections, but they were caused only by a time gap between the actual traffic light state change and the moment of refreshing of the camera image.

Also there is a problem with the picture quality in the simulator - with low graphical quality settings, the camera image would contain a poorly rendered traffic lights, which affects the detector. I even had a problem with too late detection (e.g. the detector has been recognizing the traffic light at a very short distance) due to the graphical problems.


## Rubric points checklist

* The code is built successfully and connects to the simulator - **done**
* Waypoints are published to plan Carla’s route around the track - **done**
* Controller commands are published to operate Carla’s throttle, brake, and steering - **done**
* Successfully navigate the full track more than once - **done**

A video recording of a successful lap in the simulator can be found [here](https://youtu.be/qA7bLBL-geI).

